In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option('max.columns', 100)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Resultados do Active Learning

In [16]:
df1 = pd.read_excel('sem_duplicate.xlsx')
df1 = df1[df1['target'].notnull()]
df1.shape

(272, 6)

In [17]:
df2 = pd.read_excel('active_label1.xlsx')
df2 = df2[df2['target'].notnull()]
df2['novo'] = 1
df2.shape

(124, 8)

In [18]:
df2.head()

,title,image,classification,url,reviews,p,target,novo
0,Student's Solutions Manual for Statistics: The...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Students-Solutions-Manu...,1,0.058105,0,1
1,Agile Machine Learning: Effective Machine Lear...,https://m.media-amazon.com/images/I/41YkiTR+Qv...,5.0,https://www.amazon.com/Agile-Machine-Learning-...,2,0.132468,1,1
2,Building Machine Learning and Deep Learning Mo...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Building-Machine-Learni...,2,0.077000,1,1
3,Smarter Data Science: Succeeding with Enterpri...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Smarter-Data-Science-Su...,2,0.116573,0,1
4,"Hands-On Machine Learning with C++: Build, tra...",https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Hands-Machine-Learning-...,2,0.105550,0,1


In [19]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['target'],df2['p']), roc_auc_score(df2['target'], df2['p'])

(0.3038535366991084, 0.5027173913043479)

In [20]:
df = pd.concat([df1, df2.drop('p', axis = 1)])

In [25]:
df.head()

,title,image,classification,url,reviews,target,novo
0,"Artificial Intelligence, Machine Learning, and...",https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Artificial-Intelligence...,1,1,NaN
1,Model-Based Clustering and Classification for ...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Model-Based-Clustering-...,1,1,NaN
2,Spatial Modeling in GIS and R for Earth and En...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Spatial-Modeling-Earth-...,1,1,NaN
3,Statistical Machine Learning: A Unified Framew...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Statistical-Machine-Lea...,1,1,NaN
4,Data Science for Transport: A Self-Study Guide...,https://images-na.ssl-images-amazon.com/images...,5.0,https://www.amazon.com/Data-Science-Transport-...,1,0,NaN


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [21]:
df_limpo = pd.DataFrame(index = df.index)
df_limpo['title'] = df['title']
df_limpo['novo'] = df['novo'].fillna(0)

In [22]:
df.shape

(396, 7)

In [24]:
df_limpo['novo'].value_counts()

0.0    272
1.0    124
Name: novo, dtype: int64

In [26]:
mask = ['classification','reviews']
features = df[mask]

In [27]:
y = df['target'].copy()

In [39]:
mask_train = df_limpo[:160]

mask_val = df_limpo[160:]

Xtrain, Xval = features[:160], features[160:]
ytrain, yval = y[:160], y[160:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((160, 2), (236, 2), (160,), (236,))

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[:160]['title']
title_val = df_limpo[160:]['title']

title_vec = TfidfVectorizer(min_df = 2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [42]:
from scipy.sparse import hstack

In [43]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])
Xtrain_wtitle.shape, Xval_wtitle.shape

((160, 180), (236, 180))

In [44]:
mdl = RandomForestClassifier(n_estimators=1000, random_state = 0, class_weight = 'balanced', n_jobs = 6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [45]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

In [46]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [47]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.24683581103698116, 0.5023317435082141)

# aumentar o treino

In [49]:
Xtrain2, Xval2 = features[:260], features[260:]
ytrain2, yval2 = y[:260], y[260:]

title_train2 = df_limpo[:260]['title']
title_val2 = df_limpo[260:]['title']

title_vec2 = TfidfVectorizer(min_df = 2)
title_bow_train2 = title_vec2.fit_transform(title_train2)
title_bow_val2 = title_vec2.transform(title_val2)

Xtrain_wtitle2 = hstack([Xtrain2, title_bow_train2])
Xval_wtitle2 = hstack([Xval2, title_bow_val2])

mdl2 = RandomForestClassifier(n_estimators=1000, random_state = 0, class_weight = 'balanced', n_jobs = 6)
mdl2.fit(Xtrain_wtitle2, ytrain2)

p2 = mdl2.predict_proba(Xval_wtitle2)[:,1]

average_precision_score(yval2, p2), roc_auc_score(yval2, p2)

(0.25377589358062025, 0.48014121800529574)

ap 0.17585895310304805 auc 0.48799313893653506

ap 0.24683581103698116 auc 0.5023317435082141

ap 0.25377589358062025 auc 0.48014121800529574